In [3]:
from graphviz import Digraph

# Initialize Digraph with vertical layout, reduced size
dot = Digraph(format='pdf')
dot.attr(rankdir='TB', fontsize='18', size='5.1,5.1', dpi='96')

# Define places with color sets
places = {
    "Devices": ("Devices", "DeviceSet"),
    "Attackers": ("Attackers", "AttackClass"),
    "SecuritySystem": ("SecuritySystem", "MitigatorSet"),
    "Operators": ("Operators", "OperatorSet"),
    "Alerts": ("Alerts", "AlertSet")
}

for pid, (label, colorset) in places.items():
    dot.node(pid,
             label=f'''<
             <TABLE BORDER="0" CELLBORDER="0" CELLSPACING="0">
               <TR><TD><FONT POINT-SIZE="16"><B>{colorset}</B></FONT></TD></TR>
               <TR><TD><FONT POINT-SIZE="16">{label}</FONT></TD></TR>
             </TABLE>>''',
             shape='circle')

# Define transitions: label, lambda
transitions = {
    "T1": ("CompromiseDevice", 0.06),
    "T2": ("MitigateDevice", 0.05),
    "T3": ("AdaptAttack", 0.075),
    "T4": ("GainControl", 0.080),
    "T5": ("HumanRecovery", 0.040),
    "T6": ("RaiseAlert", 0.070),
    "T7": ("TriggerMitigation", 0.090),
    "T8": ("OverloadOperator", 0.050)
}

for tid, (desc, rate) in transitions.items():
    dot.node(tid,
             label=f'''<
             <TABLE BORDER="0" CELLBORDER="0" CELLSPACING="1">
               <TR><TD><FONT POINT-SIZE="16"><B>{tid}</B></FONT></TD></TR>
               <TR><TD><FONT POINT-SIZE="16">{desc}</FONT></TD></TR>
               <TR><TD><FONT POINT-SIZE="14">λ = {rate:.3f}</FONT></TD></TR>
             </TABLE>>''',
             shape='box')

# Define arcs
arc_defs = [
    ("Attackers", "T1", "att.target = dev.d and dev.s = NORMAL"),
    ("Devices", "T1", ""),
    ("T1", "Devices", ""),

    ("Devices", "T2", "dev.s = COMPROMISED"),
    ("SecuritySystem", "T2", "mitigator ready"),
    ("T2", "Devices", ""),

    ("Attackers", "T3", "strategy failed"),
    ("T3", "Attackers", ""),

    ("Attackers", "T4", "attacker adapted"),
    ("T4", "Devices", ""),

    ("Devices", "T5", ""),
    ("Operators", "T5", "operator available and not overloaded"),
    ("T5", "Devices", ""),

    ("Devices", "T6", "#COMPROMISED(devices) > θ"),
    ("T6", "Alerts", ""),

    ("Alerts", "T7", "alert active"),
    ("T7", "SecuritySystem", ""),

    ("Alerts", "T8", "too many alerts/operator"),
    ("Operators", "T8", ""),
    ("T8", "Operators", "")
]

# Add edges with labels
for src, dst, label in arc_defs:
    dot.edge(src, dst, label=label, fontsize="14")

# Output
output_path = "./plots/cspn"
dot.render(output_path)


'plots\\cspn.pdf'